In [1]:
import requests
import time
import pandas as pd

In [2]:
LEETCODE_GRAPHQL_URL = "https://leetcode.com/graphql"

HEADERS = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.3",
    "Cookie": "LEETCODE_SESSION=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJfYXV0aF91c2VyX2lkIjoiMTY3ODMwODUiLCJfYXV0aF91c2VyX2JhY2tlbmQiOiJkamFuZ28uY29udHJpYi5hdXRoLmJhY2tlbmRzLk1vZGVsQmFja2VuZCIsIl9hdXRoX3VzZXJfaGFzaCI6ImNiMDY2NDU5NTZmNjI4YjljMTQ1Y2FhOTRlOWQzMjgwNjc4OTFiMjQ2Yzc3YjAxZmMzNWM2OTg5NGI1ZDNiZGEiLCJzZXNzaW9uX3V1aWQiOiIxODg2MWYyMSIsImlkIjoxNjc4MzA4NSwiZW1haWwiOiJrcmlzaG5hc2hhcm1hMTgwMzIwMDVAZ21haWwuY29tIiwidXNlcm5hbWUiOiJLcmlzaG5hX1NoYXJtYTE4IiwidXNlcl9zbHVnIjoiS3Jpc2huYV9TaGFybWExOCIsImF2YXRhciI6Imh0dHBzOi8vYXNzZXRzLmxlZXRjb2RlLmNvbS91c2Vycy9LcmlzaG5hX1NoYXJtYTE4L2F2YXRhcl8xNzYwMjgxNDA1LnBuZyIsInJlZnJlc2hlZF9hdCI6MTc2NzAxNDgyOSwiaXAiOiIyNDAxOjQ5MDA6ZWI4OjFlYmY6MWM0NzoxMTk1Ojg5NmE6ZjA3ZiIsImlkZW50aXR5IjoiODlkYjcyOWNmY2RjMTI5MTExZjAxN2IwZTdhYzMyNGEiLCJkZXZpY2Vfd2l0aF9pcCI6WyI1NzAxNmI5NjM1ZjcwMTJmM2I4MWJmZjNiZTllNTFjOSIsIjI0MDE6NDkwMDplYjg6MWViZjoxYzQ3OjExOTU6ODk2YTpmMDdmIl0sIl9zZXNzaW9uX2V4cGlyeSI6MTIwOTYwMH0.6dVEpW47k0_xCMRKEMxsQQmCBF0NycItsTMLZQJ95Uw; csrftoken=FgA4TeISePLWYVTFjg0Y8n23bJZ2fejg",
    "x-csrftoken":"FgA4TeISePLWYVTFjg0Y8n23bJZ2feg"
}

def run_query(payload):
    r = requests.post(
        LEETCODE_GRAPHQL_URL,
        json=payload,
        headers=HEADERS
    )
    if r.status_code != 200:
        print("STATUS:", r.status_code)
        print("RESPONSE TEXT:\n", r.text) 
        r.raise_for_status()

    return r.json()


In [3]:
def fetch_all_user_progress(run_query, page_size=50):
    all_questions = []
    skip = 0

    while True:
        payload = {
            "operationName": "userProgressQuestionList",
            "variables": {
                "filters": {
                    "skip": skip,
                    "limit": page_size
                }
            },
            "query": """
            query userProgressQuestionList($filters: UserProgressQuestionListInput) {
              userProgressQuestionList(filters: $filters) {
                totalNum
                questions {
                  frontendId
                  title
                  titleSlug
                  difficulty
                  lastSubmittedAt
                  numSubmitted
                  questionStatus
                  lastResult
                  topicTags {
                    name
                    slug
                  }
                }
              }
            }
            """
        }


        data = run_query(payload)["data"]["userProgressQuestionList"]["questions"]

        if not data:
            break

        all_questions.extend(data)
        skip += page_size

    return all_questions


In [4]:
a=fetch_all_user_progress(run_query)

In [5]:
frontendId=[]
title=[]
titleSlug=[]
difficulty=[]
lastSubmittedAt=[]
numSubmitted=[]
questionStatus=[]
lastResult=[]
topicTags=[]

for question in a:
    frontendId.append(question['frontendId'])
    title.append(question['title'])
    titleSlug.append(question['titleSlug'])
    difficulty.append(question['difficulty'])
    lastSubmittedAt.append(question['lastSubmittedAt'])
    numSubmitted.append(question['numSubmitted'])
    questionStatus.append(question['questionStatus'])
    lastResult.append(question['lastResult'])
    topicTags.append(question['topicTags'])

    
df=pd.DataFrame({
    'frontendId':frontendId,
    'title':title,
    'titleSlug':titleSlug,
    'difficulty':difficulty,
    'lastSubmittedAt':lastSubmittedAt,
    'numSubmitted':numSubmitted,
    'questionStatus':questionStatus,
    'lastResult':lastResult,
    'topicTags':topicTags
})

df.to_csv('../data/progress.csv')

In [6]:
def fetch_all_questions(run_query, page_size=500):
    all_questions = []
    skip = 0

    while True:
        payload = {
            "operationName": "problemsetQuestionListV2",
            "variables": {
                "skip": skip,
                "limit": page_size,
                "categorySlug":"all-code-essentials",
            },
            "query": """
            query problemsetQuestionListV2(
              $limit: Int,
              $searchKeyword: String,
              $skip: Int,
              $categorySlug: String
            ) {
              problemsetQuestionListV2(
                limit: $limit
                searchKeyword: $searchKeyword
                skip: $skip
                categorySlug: $categorySlug
              ) {
                questions {
                  titleSlug
                  title
                  difficulty
                  status
                }
                hasMore
              }
            }
            """
        }

        result = run_query(payload)
        data = result["data"]["problemsetQuestionListV2"]

        all_questions.extend(data["questions"])

        if not data["hasMore"]:
            break

        skip += page_size

    return all_questions


In [7]:
b=fetch_all_questions(run_query)

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [ ]:
frontendId=[]
title=[]
titleSlug=[]
difficulty=[]
lastSubmittedAt=[]
numSubmitted=[]
questionStatus=[]
lastResult=[]
topicTags=[]
status=[]


for each in b:
    frontendId.append(each['questionFrontendId'])
    title.append(each['title'])
    titleSlug.append(each['titleSlug'])
    difficulty.append(each['difficulty'])
    topicTags.append(each['topicTags'])
    status.append(each['status'])
    

df_all=pd.DataFrame({
    'frontendId':frontendId,
    'title':title,
    'titleSlug':titleSlug,
    'difficulty':difficulty,
    'topicTags':topicTags,
     'status':status,
})

df_all.to_csv('../data/questions.csv')
    
    